In [1]:
# Deciding the seen and unseen classes
import os

In [2]:
data_path = "/mnt/elk/data/valaybun/other_users/vinit/Data/ZERO_SHOT_DATASET_SHREC14"

In [3]:
# Make a folder all_classes
# all_classes = os.path.join(data_path, "all_classes")
# # if not os.path.exists(all_classes):
# #     os.makedirs(all_classes)

# # Make img and model folder inside all_classes
# img_path = os.path.join(all_classes, "img")
# model_path = os.path.join(all_classes, "model")

# if not os.path.exists(img_path):
#     os.makedirs(img_path)

# if not os.path.exists(model_path):
#     os.makedirs(model_path)

# Move all the classes from train and test folder to all_classes folder
train_path = os.path.join(data_path, "train")
test_path = os.path.join(data_path, "test")

train_path_images = os.path.join(train_path, "img")
test_path_images = os.path.join(test_path, "img")

train_path_models = os.path.join(train_path, "model")
test_path_models = os.path.join(test_path, "model")

# for class_name in os.listdir(train_path_images):
#     img_path = os.path.join(train_path_images, class_name)
#     model_path = os.path.join(train_path_models, class_name)

#     # Make a folder with class name in all_classes/img
#     new_img_path = os.path.join(all_classes, "img", class_name)

#     new_model_path = os.path.join(all_classes, "model", class_name)

#     # Use cp -r to copy the folder
#     os.system("cp -r {} {}".format(img_path, new_img_path))
#     os.system("cp -r {} {}".format(model_path, new_model_path))

# for class_name in os.listdir(test_path_images):
#     img_path = os.path.join(test_path_images, class_name)
#     model_path = os.path.join(test_path_models, class_name)

#     # Make a folder with class name in all_classes/img
#     new_img_path = os.path.join(all_classes, "img", class_name)

#     new_model_path = os.path.join(all_classes, "model", class_name)

#     # Use cp -r to copy the folder
#     os.system("cp -r {} {}".format(img_path, new_img_path))
#     os.system("cp -r {} {}".format(model_path, new_model_path))

In [4]:
from pytorch3d.structures import Meshes
from pytorch3d.renderer.mesh import Textures

/mnt/elk/data/valaybun/other_users/vinit/pytorch3d/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# For all the classes in all_classes folder, plot one of the image and all the models in a grid
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
from torchvision import transforms
from torchvision.utils import make_grid
import trimesh

In [6]:
def _load_mesh_off(model_path):
    from pytorch3d.io import IO 

    mesh = IO().load_mesh(model_path)
    verts = mesh.verts_packed()
    faces = mesh.faces_packed()

    textures = verts.new_ones(
        faces.shape[0],
        4,
        4,
        3,
    )

    return verts, faces, textures

def torch_center_and_normalize(points,p="inf"):
    """
    a helper pytorch function that normalize and center 3D points clouds 
    """
    N = points.shape[0]
    center = points.mean(0)
    if p != "fro" and p!= "no":
        scale = torch.max(torch.norm(points - center, p=float(p),dim=1))
    elif p=="fro" :
        scale = torch.norm(points - center, p=p )
    elif p=="no":
        scale = 1.0
    points = points - center.expand(N, 3)
    points = points * (1.0 / float(scale))
    return points

In [7]:
from src.models.components.mvtn import MVTN
from src.models.components.renderer import MVRenderer
from src.utils.ops import regualarize_rendered_views

In [8]:
mvtn = MVTN(8, views_config='circular',
            canonical_elevation= 30.0, 
            canonical_distance= 2.2,
            transform_distance=False, 
            input_view_noise=0.0, 
            shape_extractor='PointNet',
            screatch_feature_extractor=True)


mvrenderer = MVRenderer(nb_views=8, image_size=224,
                        pc_rendering=False, 
                        object_color='blue', 
                        background_color='white',
                        faces_per_pixel=2, points_radius=0.006,  points_per_pixel=1, 
                        light_direction="random", 
                        cull_backfaces=False)

In [9]:
# For class in all_classes/img
i_path = train_path_images
m_path = train_path_models

classes = os.listdir(i_path)
classes.sort()
classes

['airplane',
 'alarm_clock',
 'ant',
 'apple',
 'armchair',
 'ashtray',
 'axe',
 'banana',
 'barn',
 'baseball_bat',
 'basket',
 'bathtub',
 'bear_animal',
 'bed',
 'bee',
 'beer_mug',
 'bench',
 'bicycle',
 'blimp',
 'book',
 'bookshelf',
 'bowl',
 'brain',
 'bread',
 'bridge',
 'bus',
 'cabinet',
 'cake',
 'camel',
 'camera',
 'candle',
 'cannon',
 'car_sedan',
 'cell_phone',
 'chair',
 'chandelier',
 'church',
 'cigarette',
 'computer_monitor',
 'cow',
 'crab',
 'crocodile',
 'cup',
 'diamond',
 'dolphin',
 'door',
 'door_handle',
 'dragon',
 'duck',
 'ear',
 'elephant',
 'eyeglasses',
 'face',
 'fan',
 'fire_hydrant',
 'fish',
 'floor_lamp',
 'flower_with_stem',
 'flying_bird',
 'flying_saucer',
 'fork',
 'frog',
 'giraffe',
 'grapes',
 'guitar',
 'hammer',
 'hand',
 'hat',
 'head',
 'helicopter',
 'helmet',
 'horse',
 'hourglass',
 'house',
 'human_skeleton',
 'ice_cream_cone',
 'kangaroo',
 'key',
 'keyboard',
 'knife',
 'lightbulb',
 'lighter',
 'lion',
 'microphone',
 'microsco

In [36]:
for i in range(0, len(classes)//15):
    print(i)
    ran = list(range(i*15, min((i+1)*15, len(classes))))


    horizontal_concat = []

    for class_num in ran:
        class_name = classes[class_num]
        # Get the image path
        class_img_path = os.path.join(i_path, class_name)

        # Get the model path
        class_model_path = os.path.join(m_path, class_name)

        # Get the image and model paths
        img_paths = [os.path.join(class_img_path, img_name) for img_name in os.listdir(class_img_path)]
        model_paths = [os.path.join(class_model_path, model_name) for model_name in os.listdir(class_model_path)]

        row_images = []

        # Load the image and model
        img = torchvision.io.read_image(img_paths[0]).cuda()

        img = img.expand(3, img.shape[1], img.shape[2])
        row_images.append(img)
        model_count = 0

        for model_path in model_paths:
            if model_count == 10:
                break

            model_count += 1

            verts, faces, textures = _load_mesh_off(model_path)
            verts = torch_center_and_normalize(
                verts.to(torch.float), p=2)

            verts_rgb = torch.ones_like(verts)[None]
            textures = Textures(verts_rgb=verts_rgb)
            meshes = Meshes(
                verts=[verts],
                faces=[faces],
                textures=textures
            )
            points = trimesh.Trimesh(vertices=verts.numpy(
            ), faces=faces.numpy()).sample(2048, False)
            points = torch.from_numpy(points).to(torch.float)
            points = torch_center_and_normalize(points, p=2)

            c_batch_size = len(meshes)
            azim, elev, dist = mvtn(points, c_batch_size=c_batch_size)
            rendered_images, _ = mvrenderer(meshes, points, azim=azim, elev=elev, dist=dist)
            rendered_images = regualarize_rendered_views(rendered_images, 0.0, False, 0.3)

            row_images.append(rendered_images[0,3,:,:,:] * 255)

        for _ in range(10 - model_count):
            row_images.append(torch.ones_like(row_images[0]) * 255)

        horizontal_concat.append(torch.cat(row_images, dim=2))

    vertical_concat = torch.cat(horizontal_concat, dim=1)

    grid = make_grid(vertical_concat, nrow=1, padding=2, normalize=True)
    grid = grid.permute(1,2,0).detach().cpu().numpy()

    plt.figure(figsize=(20,20))
    plt.axis('off')
    plt.imshow(grid)
    plt.show()

    # save figure
    plt.savefig('all_classes' + str(i)+'.png', bbox_inches='tight', pad_inches=0) 


0
1
2
3
4
5


In [10]:
save_path = "/mnt/elk/data/valaybun/other_users/vinit/Data/ZERO_SHOT_DATASET_SHREC14/train/model_pkl"

i = 0
error_files = []
for class_name in classes:
    print(i)
    i += 1
    # Get the model path
    class_model_path = os.path.join(m_path, class_name)

    model_paths = [os.path.join(class_model_path, model_name) for model_name in os.listdir(class_model_path)]

    save_path_class = os.path.join(save_path, class_name)
    os.makedirs(save_path_class, exist_ok=True)

    for model_path in model_paths:
        try:
            verts, faces, textures = _load_mesh_off(model_path)
        except:
            error_files.append(model_path)
            print("Error in loading file: ", model_path)
            continue
        
        verts = torch_center_and_normalize(
            verts.to(torch.float), p=2)

        verts_rgb = torch.ones_like(verts)[None]
        textures = Textures(verts_rgb=verts_rgb)
        meshes = Meshes(
            verts=[verts],
            faces=[faces],
            textures=textures
        )
        points = trimesh.Trimesh(vertices=verts.numpy(
        ), faces=faces.numpy()).sample(2048, False)
        points = torch.from_numpy(points).to(torch.float)
        points = torch_center_and_normalize(points, p=2)

        c_batch_size = len(meshes)
        azim, elev, dist = mvtn(points, c_batch_size=c_batch_size)
        rendered_images, _ = mvrenderer(meshes, points, azim=azim, elev=elev, dist=dist)
        rendered_images = regualarize_rendered_views(rendered_images, 0.0, False, 0.3)

        # Remove the batch dimension
        rendered_images = rendered_images.squeeze(0)

        save_path_model = os.path.join(save_path_class, model_path.split("/")[-1].split(".")[0] + ".pkl")
        torch.save(rendered_images, os.path.join(save_path, save_path_model))


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33


In [94]:
path_1 = "/mnt/elk/data/valaybun/other_users/vinit/Data/ZERO_SHOT_DATASET_SHREC13/all_classes/model/"
path_2 = "/mnt/elk/data/valaybun/other_users/vinit/Data/ZERO_SHOT_DATASET_SHREC13/all_classes/model_pkl/"

# Check if all the models are saved with the same name in both the folders except the extension
for class_name in classes:  
    class_path_1 = os.path.join(path_1, class_name)
    class_path_2 = os.path.join(path_2, class_name)

    models_1 = os.listdir(class_path_1)
    models_2 = os.listdir(class_path_2)

    models_1 = sorted(models_1)
    models_2 = sorted(models_2)
    
    for model_1 in models_1:
        model_1_name = model_1.split(".")[0]
        model_2_name = model_1_name + ".pkl"

        assert model_2_name in models_2

# Check if the number of models in both the folders are same
for class_name in classes:
    class_path_1 = os.path.join(path_1, class_name)
    class_path_2 = os.path.join(path_2, class_name)

    models_1 = os.listdir(class_path_1)
    models_2 = os.listdir(class_path_2)

    assert len(models_1) == len(models_2)

print("All the models are saved with the same name in both the folders except the extension")

All the models are saved with the same name in both the folders except the extension


In [78]:
model_1_path = os.path.join(path_1, classes[0])
model_1_path = os.path.join(model_1_path, sorted(os.listdir(model_1_path))[0])
model_1_path

'/mnt/elk/data/valaybun/other_users/vinit/Data/ZERO_SHOT_DATASET_SHREC13/all_classes/model/bridge/m1779.off'

In [80]:
model_2_path = os.path.join(path_2, classes[0])
model_2_path = os.path.join(model_2_path, sorted(os.listdir(model_2_path))[0])
model_2_path

'/mnt/elk/data/valaybun/other_users/vinit/Data/ZERO_SHOT_DATASET_SHREC13/all_classes/model_pkl/bridge/m1779.pkl'

In [81]:
os.path.exists(model_1_path)

True

In [82]:
# Load pickle file from model_2_path

rendered_images = torch.load(model_2_path)

In [83]:
rendered_images.shape

torch.Size([8, 3, 224, 224])

In [85]:
sorted(classes)

['airplane',
 'ant',
 'axe',
 'barn',
 'bed',
 'bee',
 'beer-mug',
 'bench',
 'bicycle',
 'binoculars',
 'book',
 'brain',
 'bridge',
 'bush',
 'butterfly',
 'cabinet',
 'car_sedan',
 'castle',
 'chair',
 'church',
 'computer_monitor',
 'couch',
 'dog',
 'dolphin',
 'door',
 'dragon',
 'duck',
 'face',
 'fish',
 'floor_lamp',
 'flower_with_stem',
 'guitar',
 'hammer',
 'hand',
 'hat',
 'head',
 'helicopter',
 'horse',
 'hot_air_balloon',
 'hourglass',
 'house',
 'human-skeleton',
 'ice-cream-cone',
 'knife',
 'ladder',
 'laptop',
 'mailbox',
 'microscope',
 'motorbike',
 'palm_tree',
 'piano',
 'pickup_truck',
 'pig',
 'potted_plant',
 'rabbit',
 'race_car',
 'sailboat',
 'satellite',
 'satellite_dish',
 'screwdriver',
 'sea_turtle',
 'shark',
 'ship',
 'shoe',
 'shovel',
 'skateboard',
 'skull',
 'skyscraper',
 'snake',
 'snowman',
 'space_shuttle',
 'spider',
 'standing_bird',
 'submarine',
 'suv',
 'sword',
 'table',
 'tablelamp',
 'tent',
 'tire',
 'train',
 'tree',
 'truck',
 'tv'

In [89]:
unseen_classes = [
    'vase',
    'tv',
    'suv',
    'couch',
    'dog',
    'motorbike',
    'hot_air_balloon',
    'palm_tree',
    'castle',
    'laptop',
    'sailboat'
]

In [90]:
# Class in unseen_classes that are not in classes
set(unseen_classes) - set(classes)

set()

In [95]:
path_2 = "/mnt/elk/data/valaybun/other_users/vinit/Data/ZERO_SHOT_DATASET_SHREC13/all_classes/"

In [96]:
os.listdir(path_2)

['img', 'model_pkl', 'model']

In [97]:
data_path = "/mnt/elk/data/valaybun/other_users/vinit/Data/ZERO_SHOT_DATASET_SCHREC13_NEW/"

train_path = os.path.join(data_path, "train")
test_path = os.path.join(data_path, "test")

# In test folder put classes from unseen_classes
test_path_img = os.path.join(test_path, "img")
test_path_model = os.path.join(test_path, "model")

os.makedirs(test_path_img, exist_ok=True)
os.makedirs(test_path_model, exist_ok=True)

path_2_img = os.path.join(path_2, "img")
path_2_model = os.path.join(path_2, "model_pkl")

for class_name in unseen_classes:
    from_path_img = os.path.join(path_2_img, class_name)
    from_path_model = os.path.join(path_2_model, class_name)

    to_path_img = os.path.join(test_path_img, class_name)
    to_path_model = os.path.join(test_path_model, class_name)

    os.system("cp -r {} {}".format(from_path_img, to_path_img))
    os.system("cp -r {} {}".format(from_path_model, to_path_model))

# In train folder put classes from classes - unseen_classes
train_path_img = os.path.join(train_path, "img")
train_path_model = os.path.join(train_path, "model")

os.makedirs(train_path_img, exist_ok=True)
os.makedirs(train_path_model, exist_ok=True)

path_2_img = os.path.join(path_2, "img")
path_2_model = os.path.join(path_2, "model_pkl")

for class_name in classes:
    if class_name in unseen_classes:
        continue

    from_path_img = os.path.join(path_2_img, class_name)
    from_path_model = os.path.join(path_2_model, class_name)

    to_path_img = os.path.join(train_path_img, class_name)
    to_path_model = os.path.join(train_path_model, class_name)

    os.system("cp -r {} {}".format(from_path_img, to_path_img))
    os.system("cp -r {} {}".format(from_path_model, to_path_model))

# Check if all the classes are copied
train_path_img = os.path.join(train_path, "img")
train_path_model = os.path.join(train_path, "model")

test_path_img = os.path.join(test_path, "img")
test_path_model = os.path.join(test_path, "model")

train_classes = os.listdir(train_path_img)
test_classes = os.listdir(test_path_img)

set(classes) - set(train_classes) - set(test_classes)
    

set()

: 